# Class Activation Map

## 개요

* XAI(Explainable Artificial Intelligence, 설명 가능한 인공지능) : 모델과 추론의 신뢰성에 대한 답을 찾는 분야

### 실습 목표

1. 분류 모델의 활성화 맵을 이해합니다.
2. 다양한 활성화 맵을 구하는 방법을 알아갑니다.
3. 약지도학습(weakly supervised learning)을 이해합니다.

### 학습 내용

1. Explainable AI
2. CAM: Class Activation Map
3. Grad-CAM
4. ACoL: Adversarial Complementary Learning
5. 생각해 보기

## Explainable AI

* 모델이 잘못된 답변을 준다면 어떻게 개선할 수 있는지?
* 모델이 잘 작동한다면 왜 이런 선택을 했는지?

### Image Classification

* local feature를 추출하기 위해 CNN으로 구성된 특성 추출용 backbone network가 있음.
* backbone에서 추출된 feature map을 fully connected layer에 통과시켜 얻은 logit
* logit을 softmax activation function에 통과시켜 class에 속할 확률으 구함.

* [관련설명](https://opentutorials.org/module/3653/22995)
* logit 과 sigmoid는 서로 역함수.
* 2개의 class를 대상으로 정의하던 logit을 K개의 class를 대상으로 일반화하면 softmax 함수가 유도됨.
* softmax 함수에서 K를 2로 놓으면 sigmoid 함수로 환원.
* sigmoid 함수를 K개의 클래스를 대상으로 일반화시 softmax 함수가 유도.

$$ t = logit(y) = ln(\frac{y}{1-y}) $$
$$ y = sigmoid(t) = \frac{1}{1+exp(t)} $$

## CAM: Class Activation Map

* CAM : Class Activation Map
* 모델이 어떤 곳을 보고 어떤 클래스임을 짐작하고 있는지 확인할 수 있는 지도

### GAP(Global Average Pooling)